In [6]:
#export
import pytex.core.Operator as Operator
from pytex.helpers import varArgFunc
from pytex.core.Precedence import PRECEDENCE
from pytex.LatexInterface import LatexBasicFormatter
from pytex.core.Operator import InvertibleOperator

class Var:
    def __init__(self, name, backward_operation=None):
        self._name, self._backward_operation = name, backward_operation
    
    @property
    def backward_operation(self):
        return self._backward_operation
    
    @property
    def precedence(self):
        return (PRECEDENCE.VAR 
                if self._backward_operation is None 
                else self._backward_operation.precedence)
    
    def build(self):
        return (self._name 
                if self._backward_operation is None 
                else self._backward_operation.build())
    
    def operation(self, operator, *args):
        operation = Operation(operator)
        args = makeVar(*args)
        if isinstance(args, list):
            output = Var('output', operation(*args))
        else:
            output = Var('output', operation(args))
        return output
    
    def __invert__(self):
        if self._backward_operation is None:
            return self
        else:
            return self._backward_operation.__invert__()
        
    def __call__(self, *args):
        return self.operation(Operator.FunctionCall, self, *args)
    
    def __add__(self, other):
        return self.operation(Operator.Plus, self, other)
    def __sub__(self, other):
        return self.operation(Operator.Minus, self, other)
    def __mul__(self, other):
        return self.operation(Operator.Multiply, self, other)
    def __pow__(self, other):
        return self.operation(Operator.Power, self, other)
    def __or__(self, other):
        return self.operation(Operator.Space, self, other)
    
    def __lt__(self, other):
        return self.operation(Operator.LessThan, self, other)
    def __gt__(self, other):
        return self.operation(Operator.GreaterThan, self, other)    
    def __le__(self, other):
        return self.operation(Operator.LessThanEqual, self, other)
    def __ge__(self, other):
        return self.operation(Operator.GreaterThanEqual, self, other)
    def __eq__(self,other):
        return self.operation(Operator.Equal, self, other)
    def __ne__(self,other):
        return self.operation(~Operator.Equal, self, other)
    def __neg__(self):
        return self.operation(Operator.Negate, self)
    def __truediv__(self, other):
        return self.operation(Operator.Fraction, self, other)
    
    def root(self, other):
        return self.operation(Operator.Root, self, other)
    
def isVar(a):
    return isinstance(a, Var)

def makeVar(*args):
    return varArgFunc(lambda arg: arg if isVar(arg) else Var(str(arg)), *args)

def addEnv(s, env='gather*'):
    env = '{' + env + '}'
    return r'\begin' + env + str(s) + r'\end' + env

In [8]:
#export
class Operation:
    def __init__(self, operator):
        self._operator = operator
        self._args = None
    
    @property
    def precedence(self):
        return self._operator.precedence
    
    def __call__(self, *args):
        self._args = args
        return self
    
    def _is_invertible(self, op):
        return isinstance(op, InvertibleOperator)
    
    def build(self):
        args = [self._build_and_surround(makeVar(arg)) for arg in self._args]
        return self._operator(*args)
    
    def _build_and_surround(self, var_instance):
        return (self._get_surrounded_with_parens(var_instance.build())
               if self._should_surround_with_parens(var_instance)
               else var_instance.build())
    
    def _get_surrounded_with_parens(self, string):
        return LatexStringFormatter.surround_with_parens(string)
    
    def _should_surround_with_parens(self, var_instance):
        return (var_instance.backward_operation is not None and 
                var_instance.backward_operation.precedence < self.precedence)   
    
    def __invert__(self):
        if self._is_invertible(self._operator):
            self._operator = ~self._operator
        return self

In [9]:
from pytex.platforms.jupyter import latex
a, b = makeVar('a', 'b')
latex(~(a.root(b)))

<IPython.core.display.Latex object>

In [10]:
! ../notebook2script.py Ast.ipynb

Converted /home/default/Documents/py-latex/pytex/notebooks/core/Ast.ipynb to /home/default/Documents/py-latex/pytex/pytex/core/Ast.py


In [18]:
from pytex.platforms.jupyter import latex
latex(~Operation(Operator.Equal)(1,2))

<IPython.core.display.Latex object>

In [19]:
from pytex.platforms.jupyter import latex
a, b, c = makeVar('a','b', 'c')
latex(a + b)

<IPython.core.display.Latex object>